In [4]:
#!/usr/bin/env python
# coding: utf-8

# In[4]:


import pandas as pd

# Load the CSV file
csv_file1 = 'sata.csv'
csv_file2 = 'mpmbfinal.csv'
df1 = pd.read_csv(csv_file1)

columns_to_keep1 = ['Week Number', 'Seconds of Week', 'PRN', 'Elevation']

df1=df1[columns_to_keep1]
# ##print the first few rows
###print("Head of the file:")
###print(df1.head())

# ##print the last few rows
###print("\nTail of the file:")
###print(df1.tail())



df2 = pd.read_csv(csv_file2)

# Drop all columns except the specified ones
columns_to_keep2 = ['Week Number', 'Seconds of Week', 'PRN', 'Amplitude']
df2 = df2[columns_to_keep2]
#df.to_csv("finalmpmb.csv", sep=',', index=False, encoding='utf-8')


# ##print the first few rows
##print("Head of the file:")
##print(df2.head())

# ##print the last few rows
##print("\nTail of the file:")
##print(df2.tail())



# In[5]:


# ##print the first few rows
##print("Head of the file:")
##print(df1.head())

# ##print the last few rows
##print("\nTail of the file:")
##print(df1.tail())
import numpy as np
# Replace zero or very small values of "Amplitude" to avoid log10 issues
df2['Amplitude'] = df2['Amplitude'].replace(0, np.nan)
df2['Amplitude'] = df2['Amplitude'].fillna(df2['Amplitude'].min() / 2)

# Add a new column 'd/u' which is 20log10(Amplitude)
df2['d/u'] = 20 * np.log10(df2['Amplitude'])

# ##print the first few rows
##print("Head of the file:")
##print(df2.head())

# ##print the last few rows
##print("\nTail of the file:")
##print(df2.tail())


df1.rename(columns={'PRN Number': 'PRN'}, inplace=True)


# In[7]:


from datetime import datetime, timedelta

# Define the GPS epoch
GPS_EPOCH = datetime(1980, 1, 6)

def gps_to_utc(week, seconds):
    # Calculate the GPS time
    gps_time = GPS_EPOCH + timedelta(weeks=week, seconds=seconds)
    return gps_time

# Apply the conversion to the DataFrame
df2['UTC Time'] = df2.apply(lambda row: gps_to_utc(row['Week Number'], row['Seconds of Week']), axis=1)
df1['UTC Time'] = df1.apply(lambda row: gps_to_utc(row['Week Number'], row['Seconds of Week']), axis=1)

# Convert 'UTC Time' to a 24-hour time format
df1['Time'] = df1['UTC Time'].dt.strftime('%H:%M:%S')
df2['Time'] = df2['UTC Time'].dt.strftime('%H:%M:%S')

# ##print the first few rows to verify
##print("Head of the file with 24-hour time format:")
##print(df1.head())
##print("Head of the file with 24-hour time format:")
##print(df2.head())



# In[8]:


#grouped = df.groupby('PRN')
grouped1 = df1.sort_values(by=['PRN', 'Time']).reset_index(drop=True)
grouped2 = df2.sort_values(by=['PRN', 'Time']).reset_index(drop=True)

##print (grouped1)
##print (grouped2)


# In[9]:


# ##print unique combinations of 'PRN' and 'Time' in df1 and df2
##print("\nUnique combinations in df1:")
##print(grouped1[['PRN', 'Time']].drop_duplicates())
##print("\nUnique combinations in df2:")
##print(grouped2[['PRN', 'Time']].drop_duplicates())


# In[10]:


##print("Column names of df1:", grouped1.columns.tolist())
##print("Column names of df2:", grouped2.columns.tolist())

try:
    df_merged = grouped2.merge(grouped1[['PRN', 'Elevation', 'Time']], on=['PRN', 'Time'], how='left')
    print("Merging successful.")
except KeyError as e:
    print(f"KeyError: {e}")



#print (df_merged)


# In[12]:


#df_merged.to_csv("testingforcno.csv", sep=',', index=False, encoding='utf-8')
#grouped = df.groupby('PRN')
grouped= df_merged.sort_values(by=['PRN', 'Time']).reset_index(drop=True)
grouped= df_merged.groupby('PRN')

# #print the head and tail of each group
for prn, group in grouped:
    #print(f"\nPRN: {prn}")
    #print("Head of the group:")
    #print(group.head())
    #print("\nTail of the group:")
    #print(group.tail())


# In[13]:


import plotly.io as pio
pio.renderers.default = "iframe"


# In[15]:


import plotly.io as pio
pio.renderers.default = "iframe"

import plotly.graph_objects as go

# Plot d/u values against 24-hour time for all PRNs together
fig = go.Figure()

# Add traces for each PRN for d/u values
for prn in df_merged['PRN'].unique():
    group = df_merged[df_merged['PRN'] == prn]
    fig.add_trace(go.Scatter(x=group['Time'], y=group['d/u'], mode='lines', name=f'PRN {prn} - d/u '))

# Add traces for each PRN for C/N0 values (secondary y-axis)
for prn in df_merged['PRN'].unique():
    group = df_merged[df_merged['PRN'] == prn]
    fig.add_trace(go.Scatter(x=group['Time'], y=group['Elevation'], mode='lines', name=f'PRN {prn} - C/N0', yaxis='y2'))

# Update layout with dual y-axes
fig.update_layout(
    title='d/u values and Eleavation for all PRNs',
    xaxis_title='Time (24-hour format)',
    yaxis_title='d/u (dB)',
    yaxis2=dict(
        title='Eleavtion',
        overlaying='y',
        side='right'
    ),
    xaxis_tickformat='%H:%M'
)

# Show the figure
fig.show()


# In[16]:


import plotly.express as px
import plotly.graph_objects as go



import plotly.graph_objects as go

specific_prn = int(input("Enter a PRN to plot (e.g., 32): "))

# Plot d/u values against 24-hour time for the specific PRN
if specific_prn in grouped.groups:
   
    # Assuming you have defined 'group' and 'specific_prn'
    
    fig = go.Figure()
    
    # Add d/u values
    fig.add_trace(go.Scatter(x=group['Time'], y=group['d/u'], mode='lines', name='d/u'))
    
    # Add C/N0 values
    fig.add_trace(go.Scatter(x=group['Time'], y=group['Elevation'], mode='lines', name='C/N0', yaxis='y2'))
    
    # Update layout to have two y-axes
    fig.update_layout(
        title=f'd/u and C/N0 values for PRN {specific_prn}',
        xaxis_title='Time (24-hour format)',
        xaxis_tickformat='%H:%M:%S',
        yaxis=dict(
            title='d/u (dB)',
            titlefont=dict(
                color='blue'
            ),
            tickfont=dict(
                color='blue'
            )
        ),
        yaxis2=dict(
            title='Elevation',
            titlefont=dict(
                color='green'
            ),
            tickfont=dict(
                color='green'
            ),
            overlaying='y',
            side='right'
        )
    )
    
    fig.show()

else:
    print(f"PRN {specific_prn} not found in the dataset.")


# In[ ]:






Head of the file:
   Week Number  Seconds of Week  PRN  Elevation
0          260         200602.0   24  39.748315
1          260         200602.0   17  15.490788
2          260         200602.0   22  44.565553
3          260         200602.0    6  59.979183
4          260         200602.0   19  31.261930

Tail of the file:
        Week Number  Seconds of Week  PRN  Elevation
898146          260         287019.0   11  77.536591
898147          260         287019.0    5   9.535146
898148          260         287019.0   24  40.216654
898149          260         287019.0   20  35.520971
898150          260         287019.0   22  44.018240
Head of the file:
   Week Number  Seconds of Week  PRN  Amplitude
0          260    200602.265625   19   0.055051
1          260    200602.078125   24   0.024828
2          260    200602.281250   17   0.064084
3          260    200602.265625   22   0.030287
4          260    200602.265625    6   0.056026

Tail of the file:
        Week Number  Seconds of 

In [5]:

# Print the first few rows
print("Head of the file:")
print(df1.head())

# Print the last few rows
print("\nTail of the file:")
print(df1.tail())
import numpy as np
# Replace zero or very small values of "Amplitude" to avoid log10 issues
df2['Amplitude'] = df2['Amplitude'].replace(0, np.nan)
df2['Amplitude'] = df2['Amplitude'].fillna(df2['Amplitude'].min() / 2)

# Add a new column 'd/u' which is 20log10(Amplitude)
df2['d/u'] = 20 * np.log10(df2['Amplitude'])

# Print the first few rows
print("Head of the file:")
print(df2.head())

# Print the last few rows
print("\nTail of the file:")
print(df2.tail())


df1.rename(columns={'PRN Number': 'PRN'}, inplace=True)


Head of the file:
   Week Number  Seconds of Week  PRN  Elevation
0          260         200602.0   24  39.748315
1          260         200602.0   17  15.490788
2          260         200602.0   22  44.565553
3          260         200602.0    6  59.979183
4          260         200602.0   19  31.261930

Tail of the file:
        Week Number  Seconds of Week  PRN  Elevation
898146          260         287019.0   11  77.536591
898147          260         287019.0    5   9.535146
898148          260         287019.0   24  40.216654
898149          260         287019.0   20  35.520971
898150          260         287019.0   22  44.018240
Head of the file:
   Week Number  Seconds of Week  PRN  Amplitude        d/u
0          260    200602.265625   19   0.055051 -25.184654
1          260    200602.078125   24   0.024828 -32.101296
2          260    200602.281250   17   0.064084 -23.865009
3          260    200602.265625   22   0.030287 -30.374897
4          260    200602.265625    6   0.056

In [7]:
from datetime import datetime, timedelta

# Define the GPS epoch
GPS_EPOCH = datetime(1980, 1, 6)

def gps_to_utc(week, seconds):
    # Calculate the GPS time
    gps_time = GPS_EPOCH + timedelta(weeks=week, seconds=seconds)
    return gps_time

# Apply the conversion to the DataFrame
df2['UTC Time'] = df2.apply(lambda row: gps_to_utc(row['Week Number'], row['Seconds of Week']), axis=1)
df1['UTC Time'] = df1.apply(lambda row: gps_to_utc(row['Week Number'], row['Seconds of Week']), axis=1)

# Convert 'UTC Time' to a 24-hour time format
df1['Time'] = df1['UTC Time'].dt.strftime('%H:%M:%S')
df2['Time'] = df2['UTC Time'].dt.strftime('%H:%M:%S')

# Print the first few rows to verify
print("Head of the file with 24-hour time format:")
print(df1.head())
print("Head of the file with 24-hour time format:")
print(df2.head())



Head of the file with 24-hour time format:
   Week Number  Seconds of Week  PRN  Elevation            UTC Time      Time
0          260         200602.0   24  39.748315 1985-01-01 07:43:22  07:43:22
1          260         200602.0   17  15.490788 1985-01-01 07:43:22  07:43:22
2          260         200602.0   22  44.565553 1985-01-01 07:43:22  07:43:22
3          260         200602.0    6  59.979183 1985-01-01 07:43:22  07:43:22
4          260         200602.0   19  31.261930 1985-01-01 07:43:22  07:43:22
Head of the file with 24-hour time format:
   Week Number  Seconds of Week  PRN  Amplitude        d/u  \
0          260    200602.265625   19   0.055051 -25.184654   
1          260    200602.078125   24   0.024828 -32.101296   
2          260    200602.281250   17   0.064084 -23.865009   
3          260    200602.265625   22   0.030287 -30.374897   
4          260    200602.265625    6   0.056026 -25.032170   

                    UTC Time      Time  
0 1985-01-01 07:43:22.265625  07

In [8]:
#grouped = df.groupby('PRN')
grouped1 = df1.sort_values(by=['PRN', 'Time']).reset_index(drop=True)
grouped2 = df2.sort_values(by=['PRN', 'Time']).reset_index(drop=True)

print (grouped1)
print (grouped2)

        Week Number  Seconds of Week  PRN  Elevation            UTC Time  \
0               260         262653.0    2   8.390163 1985-01-02 00:57:33   
1               260         262654.0    2   8.392177 1985-01-02 00:57:34   
2               260         262655.0    2   8.394189 1985-01-02 00:57:35   
3               260         262656.0    2   8.396202 1985-01-02 00:57:36   
4               260         262657.0    2   8.398219 1985-01-02 00:57:37   
...             ...              ...  ...        ...                 ...   
898146          260         252217.0   32   3.102991 1985-01-01 22:03:37   
898147          260         252218.0   32   3.100060 1985-01-01 22:03:38   
898148          260         252219.0   32   3.097124 1985-01-01 22:03:39   
898149          260         252220.0   32   3.094189 1985-01-01 22:03:40   
898150          260         252221.0   32   3.091255 1985-01-01 22:03:41   

            Time  
0       00:57:33  
1       00:57:34  
2       00:57:35  
3       00:

In [9]:
# Print unique combinations of 'PRN' and 'Time' in df1 and df2
print("\nUnique combinations in df1:")
print(grouped1[['PRN', 'Time']].drop_duplicates())
print("\nUnique combinations in df2:")
print(grouped2[['PRN', 'Time']].drop_duplicates())



Unique combinations in df1:
        PRN      Time
0         2  00:57:33
1         2  00:57:34
2         2  00:57:35
3         2  00:57:36
4         2  00:57:37
...     ...       ...
898146   32  22:03:37
898147   32  22:03:38
898148   32  22:03:39
898149   32  22:03:40
898150   32  22:03:41

[897949 rows x 2 columns]

Unique combinations in df2:
        PRN      Time
0         2  00:57:34
1         2  00:57:35
2         2  00:57:36
3         2  00:57:37
4         2  00:57:38
...     ...       ...
897899   32  22:03:35
897900   32  22:03:36
897901   32  22:03:37
897902   32  22:03:38
897903   32  22:03:40

[897754 rows x 2 columns]


In [10]:
print("Column names of df1:", grouped1.columns.tolist())
print("Column names of df2:", grouped2.columns.tolist())

try:
    df_merged = grouped2.merge(grouped1[['PRN', 'Elevation', 'Time']], on=['PRN', 'Time'], how='left')
    print("Merging successful.")
except KeyError as e:
    print(f"KeyError: {e}")

Column names of df1: ['Week Number', 'Seconds of Week', 'PRN', 'Elevation', 'UTC Time', 'Time']
Column names of df2: ['Week Number', 'Seconds of Week', 'PRN', 'Amplitude', 'd/u', 'UTC Time', 'Time']
Merging successful.


In [11]:
print (df_merged)

        Week Number  Seconds of Week  PRN  Amplitude        d/u  \
0               260    262654.000000    2   0.047369 -26.490112   
1               260    262655.000000    2   0.068760 -23.253272   
2               260    262656.000000    2   0.056255 -24.996815   
3               260    262657.000000    2   0.055753 -25.074669   
4               260    262658.000000    2   0.038389 -28.315828   
...             ...              ...  ...        ...        ...   
898253          260    252215.296875   32   0.059791 -24.467313   
898254          260    252216.296875   32   0.073434 -22.682015   
898255          260    252217.296875   32   0.058562 -24.647708   
898256          260    252218.296875   32   0.198071 -14.063591   
898257          260    252220.296875   32   0.074706 -22.532881   

                         UTC Time      Time  Elevation  
0      1985-01-02 00:57:34.000000  00:57:34   8.392177  
1      1985-01-02 00:57:35.000000  00:57:35   8.394189  
2      1985-01-02 00:57:

In [12]:
#df_merged.to_csv("testingforcno.csv", sep=',', index=False, encoding='utf-8')
#grouped = df.groupby('PRN')
grouped = df_merged.sort_values(by=['PRN', 'Time']).reset_index(drop=True)
grouped = df_merged.groupby('PRN')

# Print the head and tail of each group
for prn, group in grouped:
    print(f"\nPRN: {prn}")
    print("Head of the group:")
    print(group.head())
    print("\nTail of the group:")
    print(group.tail())


PRN: 2
Head of the group:
   Week Number  Seconds of Week  PRN  Amplitude        d/u  \
0          260         262654.0    2   0.047369 -26.490112   
1          260         262655.0    2   0.068760 -23.253272   
2          260         262656.0    2   0.056255 -24.996815   
3          260         262657.0    2   0.055753 -25.074669   
4          260         262658.0    2   0.038389 -28.315828   

             UTC Time      Time  Elevation  
0 1985-01-02 00:57:34  00:57:34   8.392177  
1 1985-01-02 00:57:35  00:57:35   8.394189  
2 1985-01-02 00:57:36  00:57:36   8.396202  
3 1985-01-02 00:57:37  00:57:37   8.398219  
4 1985-01-02 00:57:38  00:57:38   8.400236  

Tail of the group:
       Week Number  Seconds of Week  PRN  Amplitude        d/u  \
35364          260    256645.296875    2   0.076189 -22.362129   
35365          260    256646.296875    2   0.059616 -24.492671   
35366          260    256647.296875    2   0.068751 -23.254409   
35367          260    256648.296875    2   0.1

In [13]:
import plotly.io as pio
pio.renderers.default = "iframe"


In [15]:
import plotly.io as pio
pio.renderers.default = "iframe"

import plotly.graph_objects as go

# Plot d/u values against 24-hour time for all PRNs together
fig = go.Figure()

# Add traces for each PRN for d/u values
for prn in df_merged['PRN'].unique():
    group = df_merged[df_merged['PRN'] == prn]
    fig.add_trace(go.Scatter(x=group['Time'], y=group['d/u'], mode='lines', name=f'PRN {prn} - d/u '))

# Add traces for each PRN for C/N0 values (secondary y-axis)
for prn in df_merged['PRN'].unique():
    group = df_merged[df_merged['PRN'] == prn]
    fig.add_trace(go.Scatter(x=group['Time'], y=group['Elevation'], mode='lines', name=f'PRN {prn} - C/N0', yaxis='y2'))

# Update layout with dual y-axes
fig.update_layout(
    title='d/u values and Eleavation for all PRNs',
    xaxis_title='Time (24-hour format)',
    yaxis_title='d/u (dB)',
    yaxis2=dict(
        title='Eleavtion',
        overlaying='y',
        side='right'
    ),
    xaxis_tickformat='%H:%M'
)

# Show the figure
fig.show()


In [16]:
import plotly.express as px
import plotly.graph_objects as go



import plotly.graph_objects as go

specific_prn = int(input("Enter a PRN to plot (e.g., 32): "))

# Plot d/u values against 24-hour time for the specific PRN
if specific_prn in grouped.groups:
   
    # Assuming you have defined 'group' and 'specific_prn'
    
    fig = go.Figure()
    
    # Add d/u values
    fig.add_trace(go.Scatter(x=group['Time'], y=group['d/u'], mode='lines', name='d/u'))
    
    # Add C/N0 values
    fig.add_trace(go.Scatter(x=group['Time'], y=group['Elevation'], mode='lines', name='C/N0', yaxis='y2'))
    
    # Update layout to have two y-axes
    fig.update_layout(
        title=f'd/u and C/N0 values for PRN {specific_prn}',
        xaxis_title='Time (24-hour format)',
        xaxis_tickformat='%H:%M:%S',
        yaxis=dict(
            title='d/u (dB)',
            titlefont=dict(
                color='blue'
            ),
            tickfont=dict(
                color='blue'
            )
        ),
        yaxis2=dict(
            title='Elevation',
            titlefont=dict(
                color='green'
            ),
            tickfont=dict(
                color='green'
            ),
            overlaying='y',
            side='right'
        )
    )
    
    fig.show()

else:
    print(f"PRN {specific_prn} not found in the dataset.")


Enter a PRN to plot (e.g., 32):  32
